In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

plt.style.use('fivethirtyeight')
sns.set_context("notebook")

Now let's play around a bit with the large baby names dataset we saw in lecture 1. We'll start by loading that dataset from the social security administration's website.

To keep the data small enough to avoid crashing datahub, we're going to look at only California rather than looking at the national dataset.

In [3]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'STATE.CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.sample(5)

,State,Sex,Year,Name,Count
8446,CA,F,1927,Mary,1816
40173,CA,F,1958,Glinda,6
66525,CA,F,1972,Octavia,9
361066,CA,M,2007,Victor,1033
335333,CA,M,1997,Jonathon,173


## Goal 1: Find the most popular baby name in California in 2018

In [20]:
most_popular = babynames.loc[[babynames['Count'].idxmax()]]
most_popular

,State,Sex,Year,Name,Count
268041,CA,M,1957,Michael,8260


## Goal 2: Baby names that start with j. 

In [22]:
j_babies = babynames[babynames['Name'].str.startswith('J')]
j_babies 

,State,Sex,Year,Name,Count
16,CA,F,1910,Josephine,66
44,CA,F,1910,Jean,35
46,CA,F,1910,Jessie,32
59,CA,F,1910,Julia,28
66,CA,F,1910,Juanita,25
...,...,...,...,...,...
407245,CA,M,2022,Jibreel,5
407246,CA,M,2022,Joseangel,5
407247,CA,M,2022,Josejulian,5
407248,CA,M,2022,Juelz,5


## Goal 3: Sort names by their length.

Approach 1a: Create a new series of only the lengths. Then add that series to the dataframe as a column. Then sort by that column. Then drop that column.

In [28]:
lengths = babynames['Name'].apply(len)
babynames['lengths'] = lengths
babynames = babynames.sort_values('lengths')
babynames = babynames.drop('lengths',axis = 1)
babynames

,State,Sex,Year,Name,Count
326570,CA,M,1993,An,8
292150,CA,M,1976,Al,13
252556,CA,M,1937,Al,21
401470,CA,M,2020,Jr,5
260022,CA,M,1948,Ed,43
...,...,...,...,...,...
339472,CA,M,1998,Franciscojavier,6
327358,CA,M,1993,Johnchristopher,5
334166,CA,M,1996,Franciscojavier,8
102505,CA,F,1986,Mariadelosangel,5


Approach 1b: Same as 1a, but use str.len() to generate the lengths of the strings.

In [32]:
lengths = babynames['Name'].str.len()
babynames['lengths'] = lengths
babynames.sort_values('lengths')
babynames.drop('lengths',axis=1)

,State,Sex,Year,Name,Count
326570,CA,M,1993,An,8
292150,CA,M,1976,Al,13
252556,CA,M,1937,Al,21
401470,CA,M,2020,Jr,5
260022,CA,M,1948,Ed,43
...,...,...,...,...,...
339472,CA,M,1998,Franciscojavier,6
327358,CA,M,1993,Johnchristopher,5
334166,CA,M,1996,Franciscojavier,8
102505,CA,F,1986,Mariadelosangel,5


Approach 2: Generate an index that is in the order we want. Pass that index to loc.

In [36]:
babynames.loc[babynames["Name"].str.len().sort_values().index].head(5)

,State,Sex,Year,Name,Count,lengths
326570,CA,M,1993,An,8,2
384992,CA,M,2015,Ty,50,2
394985,CA,M,2018,Om,9,2
80263,CA,F,1978,My,6,2
264928,CA,M,1953,Ty,6,2


How does this work exactly? Let's break it into pieces.

In [37]:
lengths_sorted_by_length = babynames["Name"].str.len().sort_values()
lengths_sorted_by_length.head(5)

326570    2
384992    2
394985    2
80263     2
264928    2
Name: Name, dtype: int64

In [39]:
index_sorted_by_length = lengths_sorted_by_length.index
index_sorted_by_length

Index([326570, 384992, 394985,  80263, 264928, 245355,  74308, 166385, 395524,
       395722,
       ...
       321913, 337301, 312543, 316886, 327472, 313977, 344440, 102505, 337477,
       321792],
      dtype='int64', length=407428)

In [40]:
# now pass the index to loc. This is yet another way 
# that loc can be used that we did not discuss in lecture!
babynames.loc[index_sorted_by_length].head(5)

,State,Sex,Year,Name,Count,lengths
326570,CA,M,1993,An,8,2
384992,CA,M,2015,Ty,50,2
394985,CA,M,2018,Om,9,2
80263,CA,F,1978,My,6,2
264928,CA,M,1953,Ty,6,2
